In [8]:
##Paso1: importar librerias necesarias 
import pandas as pd
import numpy as np
from skimpy import skim
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

In [14]:
##Paso 2: Cargar la base de datos y ver las primeras filas con head 
df = pd.read_csv(r'C:\Users\RAC-BELLABFES\Desktop\proyectoICAM\ReporteICAM.csv') 
df.head()

,ano,muestreo,departamento,estacion,coduer,sustrato,latitud,longitud,ca_cla,ca_cte,...,hdd,no3,od,ph,po4,sst,icam,icam_calificacion,num_variables,confianza
0,2018,2018--2,ATLANTICO,Bocas de Ceniza (B),C08001003,Agua Estuarina,11.104833,-74.853778,84.38,23.94,...,0.24,234.0,5.41,7.31,42.0,242.0,36.00,INADECUADA,7,0.87
1,2018,2018--1,ATLANTICO,C. Balboa,C08002011,Agua Estuarina,10.984842,-74.965589,21.19,93.86,...,0.18,-1.0,8.76,8.31,7.0,49.9,67.46,ACEPTABLE,6,0.78
2,2018,2018--2,ATLANTICO,C. Balboa,C08002011,Agua Estuarina,10.984842,-74.965589,47.18,82.66,...,0.16,12.9,9.42,8.25,-1.0,134.0,70.44,ADECUADA,6,0.75
3,2018,2018--1,ATLANTICO,C. Mallorquin-Playa,C08003042,Agua Estuarina,11.036472,-74.859806,22.16,NaN,...,0.47,6870.0,6.30,7.88,407.0,42.3,20.43,PESIMA,6,0.73
4,2018,2018--2,ATLANTICO,C. Mallorquin-Playa,C08003042,Agua Estuarina,11.036472,-74.859806,39.76,12.26,...,0.31,497.0,7.25,7.93,469.0,51.9,19.55,PESIMA,7,0.87


In [15]:
##Paso3: haremos exploracion de datos
  # Shape muestra el tamaño del dataset
df.shape
df.info


<bound method DataFrame.info of        ano muestreo     departamento             estacion     coduer  \
0     2018  2018--2        ATLANTICO  Bocas de Ceniza (B)  C08001003   
1     2018  2018--1        ATLANTICO            C. Balboa  C08002011   
2     2018  2018--2        ATLANTICO            C. Balboa  C08002011   
3     2018  2018--1        ATLANTICO  C. Mallorquin-Playa  C08003042   
4     2018  2018--2        ATLANTICO  C. Mallorquin-Playa  C08003042   
...    ...      ...              ...                  ...        ...   
1710  2024  2024--1  VALLE DEL CAUCA         H. Palm View  P76001008   
1711  2024  2024--1  VALLE DEL CAUCA       Hotel Medellin  P76001005   
1712  2024  2024--1  VALLE DEL CAUCA             La Barra  P76001002   
1713  2024  2024--1  VALLE DEL CAUCA      P. Juan de Dios  P76001025   
1714  2024  2024--1  VALLE DEL CAUCA           P. Magüipi  P76002027   

            sustrato    latitud   longitud  ca_cla  ca_cte  ...    hdd  \
0     Agua Estuarina  11.1048

In [17]:
df.columns

Index(['ano', 'muestreo', 'departamento', 'estacion', 'coduer', 'sustrato',
       'latitud', 'longitud', 'ca_cla', 'ca_cte', 'ca_cot', 'ca_hdd', 'ca_no3',
       'ca_od', 'ca_ph', 'ca_po4', 'ca_sst', 'eval_cla', 'eval_cte',
       'eval_cot', 'eval_hdd', 'eval_no3', 'eval_od', 'eval_ph', 'eval_sst',
       'eval_po4', 'cot', 'cla', 'cte', 'hdd', 'no3', 'od', 'ph', 'po4', 'sst',
       'icam', 'icam_calificacion', 'num_variables', 'confianza'],
      dtype='object')

In [16]:
   #dtypes muestra los tipos de datos
""" od :oxigeno disuelto
ph: potencial de hidrogeno
N-NO3: nitratos

"""

df.dtypes

ano                    int64
muestreo              object
departamento          object
estacion              object
coduer                object
sustrato              object
latitud              float64
longitud             float64
ca_cla               float64
ca_cte               float64
ca_cot               float64
ca_hdd               float64
ca_no3               float64
ca_od                float64
ca_ph                float64
ca_po4               float64
ca_sst               float64
eval_cla              object
eval_cte              object
eval_cot              object
eval_hdd              object
eval_no3              object
eval_od               object
eval_ph               object
eval_sst              object
eval_po4              object
cot                  float64
cla                  float64
cte                  float64
hdd                  float64
no3                  float64
od                   float64
ph                   float64
po4                  float64
sst           

In [20]:
icam = df.rename(columns={
    'ano':'año', 
    'muestreo':'muestreo', 
    'departamento':'departamento', 
    'estacion':'estacion', 
    'coduer':'codigo_estacion', 
    'sustrato':'sustrato',
    'latitud':'latitud', 
    'longitud':'longitud', 
    'ca_cla':'calidad_clorofilaa', 
    'ca_cte':'calidad_coliformes_termotolerantes', 
    'ca_cot':'calidad_carbono_organico_total', 
    'ca_hdd':'calidad_hdd', 
    'ca_no3':'calidad_nitratos',
    'ca_od':'calidad_oxigeno_disuelto', 
    'ca_ph':'calidad_ph', 
    'ca_po4':'calidad_ortofosfatos', 
    'ca_sst':'calidad_sst', 
    'eval_cla':'evaluacion_clorofilaa', 
    'eval_cte':'evaluacion_coliformes_termotolerantes',
    'eval_cot':'evaluacion_carbono_organico_total', 
    'eval_hdd':'evaluacion_hdd', 
    'eval_no3':'evaluacion_nitratos', 
    'eval_od':'evaluacion_oxigeno_disuelto', 
    'eval_ph':'evaluacion_ph', 
    'eval_sst':'evaluacion_sst',
    'eval_po4':'evaluacion_ortofosfatos', 
    'cot':'carbono_organico_total', 
    'cla':'clorofilaa', 
    'cte':'coliformes_termotolerantes', 
    'hdd':'hidrocarburos_disueltos_dispersos', 
    'no3':'nitratos', 
    'od':'oxigeno_disuelto', 
    'ph':'ph', 
    'po4':'ortofosfatos', 
    'sst':'solidos_suspendidos_totales',
    'icam':'icam', 
    'icam_calificacion':'icam_calificacion', 
    'num_variables':'año', 
    'confianza':'confianza_resultado',}
                                 , inplace = True)

In [21]:
df.columns

Index(['año', 'muestreo', 'departamento', 'estacion', 'codigo_estacion',
       'sustrato', 'latitud', 'longitud', 'calidad_clorofilaa',
       'calidad_coliformes_termotolerantes', 'calidad_carbono_organico_total',
       'calidad_hdd', 'calidad_nitratos', 'calidad_oxigeno_disuelto',
       'calidad_ph', 'calidad_ortofosfatos', 'calidad_sst',
       'evaluacion_clorofilaa', 'evaluacion_coliformes_termotolerantes',
       'evaluacion_carbono_organico_total', 'evaluacion_hdd',
       'evaluacion_nitratos', 'evaluacion_oxigeno_disuelto', 'evaluacion_ph',
       'evaluacion_sst', 'evaluacion_ortofosfatos', 'carbono_organico_total',
       'clorofilaa', 'coliformes_termotolerantes',
       'hidrocarburos_disueltos_dispersos', 'nitratos', 'oxigeno_disuelto',
       'ph', 'ortofosfatos', 'solidos_suspendidos_totales', 'icam',
       'icam_calificacion', 'año', 'confianza_resultado'],
      dtype='object')

In [ ]:
  ## describe exploramos estadistica descriptiva
icam.describe()

,ano,latitud,longitud,ca_cla,ca_cte,ca_cot,ca_hdd,ca_no3,ca_od,ca_ph,...,cte,hdd,no3,od,ph,po4,sst,icam,num_variables,confianza
count,1715.000000,1715.000000,1715.000000,1088.000000,1373.000000,300.000000,837.000000,1032.000000,1695.000000,1695.000000,...,1.484000e+03,1280.000000,1620.000000,1695.000000,1695.000000,1651.00000,1671.000000,1715.000000,1715.000000,1715.000000
mean,2020.723032,9.742268,-76.225471,70.347371,68.799876,63.503567,95.413023,67.105601,80.872165,86.949038,...,1.607646e+04,-0.041884,31.256723,6.543776,8.151309,15.61472,36.450916,41.297708,5.690379,0.722047
std,2.041209,3.063488,2.709086,26.258699,33.719916,26.197492,6.036562,32.422691,14.943570,10.453862,...,4.184946e+05,0.874640,194.658194,1.309667,0.315319,94.31570,69.125339,37.206620,1.314268,0.157941
min,2018.000000,1.642861,-81.732481,21.190000,0.610000,14.600000,33.330000,0.180000,1.000000,7.470000,...,-1.000000e+00,-1.000000,-1.000000,0.360000,6.020000,-1.00000,-1.000000,-1.000000,1.000000,0.120000
25%,2019.000000,9.404033,-77.411713,49.820000,45.020000,40.452500,95.010000,47.825000,77.260000,84.470000,...,4.500000e+00,-1.000000,-1.000000,6.060000,8.070000,-1.00000,7.945000,-1.000000,5.000000,0.630000
50%,2021.000000,11.021056,-75.649731,78.370000,84.510000,64.305000,97.710000,85.955000,83.140000,88.780000,...,4.900000e+01,0.120000,4.020000,6.480000,8.172000,3.32000,17.600000,45.840000,6.000000,0.750000
75%,2022.000000,11.319567,-74.223054,94.475000,98.540000,90.005000,98.800000,91.315000,89.040000,92.380000,...,5.400000e+02,0.322500,20.175000,6.985000,8.280000,8.70500,38.200000,78.835000,7.000000,0.870000
max,2024.000000,13.385900,-71.981544,100.000000,100.000000,98.620000,99.240000,99.630000,100.000000,98.760000,...,1.600000e+07,8.600000,6870.000000,20.000000,9.930000,2100.00000,1604.000000,95.930000,8.000000,1.000000


In [12]:
  ##isnull muestra los valores nulos
df.isnull().sum()

ano                     0
muestreo                0
departamento            0
estacion                0
coduer                  0
sustrato                0
latitud                 0
longitud                0
ca_cla                627
ca_cte                342
ca_cot               1415
ca_hdd                878
ca_no3                683
ca_od                  20
ca_ph                  20
ca_po4                605
ca_sst                 60
eval_cla              627
eval_cte              342
eval_cot             1415
eval_hdd              878
eval_no3              683
eval_od                20
eval_ph                20
eval_sst               60
eval_po4              605
cot                  1376
cla                   581
cte                   231
hdd                   435
no3                    95
od                     20
ph                     20
po4                    64
sst                    44
icam                    0
icam_calificacion       0
num_variables           0
confianza   

In [14]:
  ## columns muestra los nombres de las columnas 
df.columns

Index(['ano', 'muestreo', 'departamento', 'estacion', 'coduer', 'sustrato',
       'latitud', 'longitud', 'ca_cla', 'ca_cte', 'ca_cot', 'ca_hdd', 'ca_no3',
       'ca_od', 'ca_ph', 'ca_po4', 'ca_sst', 'eval_cla', 'eval_cte',
       'eval_cot', 'eval_hdd', 'eval_no3', 'eval_od', 'eval_ph', 'eval_sst',
       'eval_po4', 'cot', 'cla', 'cte', 'hdd', 'no3', 'od', 'ph', 'po4', 'sst',
       'icam', 'icam_calificacion', 'num_variables', 'confianza'],
      dtype='object')

In [ ]:
##Paso4 limpieza de datos